In [1]:
import torch
import h5py
import sys
import subprocess
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.distributions import Normal
import numpy as np
import argparse
import time
from helpers import timeSince, asMinutes
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from baseline_model import *


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
data_fn = '/n/data_02/Basset/data/mini_roadmap.h5'

start = time.time()
print("Reading data from file {}".format(data_fn))
data = h5py.File(data_fn)

#train = torch.utils.data.TensorDataset(torch.CharTensor(data['train_in']), torch.CharTensor(data['train_out']))
val = torch.utils.data.TensorDataset(torch.CharTensor(data['valid_in']), torch.CharTensor(data['valid_out']))
test = torch.utils.data.TensorDataset(torch.CharTensor(data['test_in']), torch.CharTensor(data['test_out']))
#train_loader = torch.utils.data.DataLoader(train, batch_size=args.batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=100, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size=100, shuffle=False)
print("Dataloaders generated {}".format( timeSince(start) ))
start = None


Reading data from file /n/data_02/Basset/data/mini_roadmap.h5
Dataloaders generated 0m 39s


In [3]:
model_fn = 'stupid.pkl'
print("Import model from {}".format(model_fn))

model = Basset()
model.load_state_dict(torch.load(model_fn))
print("Done")

Import model from stupid.pkl
Done


In [4]:
val_tensor = to_one_hot(torch.FloatTensor(data['valid_in'][()].astype('uint8')), n_dims=4).permute(0,3,1,2).squeeze().float()

In [27]:
model.cuda()
model.eval()
val_tensor = val_tensor.cuda()

val_set = model( val_tensor[100:200] )

In [13]:
np.set_printoptions(threshold=np.nan)

In [42]:
inputs, targets = val_tensor[100:200], data['valid_out'][100:200][()]

In [43]:
inputs.shape

torch.Size([100, 4, 600])

In [44]:
threshold = 0.5
preds = (F.sigmoid(model( inputs )).cpu().data.numpy() > threshold).astype(int)
matches = np.sum(preds == targets)
tru_pos = np.sum(matches == targets) / np.sum(targets)
fls_pos = np.sum( (not matches) == targets ) / np.sum(targets == 0)


In [54]:
np.sum(matches == targets)

0

In [28]:
threshold = 0.5
for inputs, targets in val_loader:
    preds = (F.sigmoid(model( inputs )).cpu().data.numpy() > threshold).astype(int)
    matches = np.sum(preds == targets)
    tru_pos = np.sum(matches == targets) / np.sum(targets)
    fls_pos = np.sum( (not matches) == targets ) / np.sum(targets == 0)


pred
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
val
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
pred
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0